# Эх хэлний боловсруулалтад гүнзгий сургалт ашиглах нь

## Мэдээллийн хайлт

In [ ]:
import re  # Текстээс тодорхой тэмдэгтүүдийг салгах, цэвэрлэх
import itertools  # Давталт, комбинаторик үйлдэлд ашиглах

import nltk  # Эх хэлний боловсруулалтын сан
nltk.download('stopwords')
from nltk.corpus import stopwords  # Зогсоох үгсийн жагсаалт
from nltk.tokenize import sent_tokenize, word_tokenize  # Өгүүлбэр болон үг болгон хуваах
from nltk.tokenize.toktok import ToktokTokenizer  # Өөр нэг tokenize арга
tokenizer = ToktokTokenizer()  # Tokenizer-ийг ашиглахад бэлэн

# Үгсийн моделиуд болон вектор боловсруулалт
!pip install gensim
import gensim
from gensim.models import Word2Vec

# Тоон утгатай массив, векторын алгебр
import numpy as np

# Косинусын төстэй байдлыг тооцоход ашиглагдах сан
import scipy
from scipy import spatial

# Англи хэлний зогсоох үгсийг тодорхойлж хадгалах
stopword_list = nltk.corpus.stopwords.words('english')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 74.0 MB/s eta 0:00:00


In [ ]:
# Жишээ баримтуудыг тодорхойлох
Doc1 = ["With the Union cabinet approving the amendments to the Motor Vehicles Act, 2016, those caught for drunken driving will have to have really deep pockets, as the fine payable in court has been enhanced to Rs 10,000 for first-time offenders."]

Doc2 = ["Natural language processing (NLP) is an area of computer science and artificial intelligence concerned with the interactions between computers and human (natural) languages, in particular how to program computers to process and analyze large amounts of natural language data."]

Doc3 = ["He points out that public transport is very good in Mumbai and New Delhi, where there is a good network of suburban and metro rail systems."]

Doc4 = ["But the man behind the wickets at the other end was watching just as keenly. With an affirmative nod from Dhoni, India captain Rohit Sharma promptly asked for a review. Sure enough, the ball would have clipped the top of middle and leg."]

# Бүх баримтыг нэгтгэж нэг жагсаалт болгон хадгалах
fin= Doc1+Doc2+Doc3+Doc4


In [ ]:
# Урьдчилан сургасан Word2Vec моделиудыг ачаалах
# ‘GoogleNews-vectors-negative300.bin’ файлыг татаж, замыг зөв зааж өгөх шаардлагатай
!pip install gensim
import gensim
model = gensim.models.KeyedVectors.load_word2vec_format(
			'GoogleNews-vectors-negative300.bin', # Моделийн байрлал
			binary=True	# Урьдчилан сургасан бинар форматтай файл ачаалах
)


In [ ]:
# Урьдчилан боловсруулах функц: Зогсоох үгс устгах, тэмдэгтүүд цэвэрлэх
def remove_stopwords(text, is_lower_case=False):
    # Тусгай тэмдэгтүүдийг устгах
    pattern = r'[^a-zA-z0-9\s]'
    text = re.sub(pattern, '', ''.join(text))

    # Үгийг токенжуулах
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]

    # Зогсоох үгсийг устгах
    if is_lower_case:
        filtered_tokens = [token for token in tokens if token not in stopword_list]
    else:
        filtered_tokens = [token for token in tokens if token.lower() not in stopword_list]

    # Үгсийг нэг мөрт текст болгон буцаах
    filtered_text = ' '.join(filtered_tokens)
    return filtered_text


In [ ]:
# Үгийн Word2Vec оруулга (embedding) авах функц
def get_embedding(word):
    if word in model.key_to_index: # Үг загварт байгаа эсэхийг шалгах
        return model[word]	# Үгийн векторыг буцаах
    else:
        return np.zeros(300)	# Үг байхгүй бол 300 хэмжээст тэг вектор буцаах

In [ ]:
import nltk
nltk.download('punkt_tab')

# Баримт бүрийн дундаж векторыг тооцоолох
out_dict =  {}

for sen in fin:
    # Баримтаас stopword-ыг цэвэрлэж үгсийг салгах
    tokens = nltk.word_tokenize(remove_stopwords(sen))

    # Үгийн Word2Vec векторуудыг авах
    word_vectors = [get_embedding(x) for x in tokens]

    # Бүх үгийн векторуудын дунджийг тооцоолох
    average_vector = np.mean(np.array(word_vectors), axis=0)

    # Баримт болон түүний дундаж векторыг хадгалах
    out_dict[sen] = average_vector


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [ ]:
# Асуулга ба баримтын вектор хоорондын ижил төсөөг тооцоолох функц
def get_sim(query_embedding, average_vector_doc):
    sim = [(1 - scipy.spatial.distance.cosine(query_embedding, average_vector_doc))]
    return sim

In [ ]:
# Бүх баримтын ижил төсөөт байдалд эрэмбэ тогтоох
def Ranked_documents(query):
    # query-ийн дундаж вектор дүрслэл үүсгэх
    query_words =  (np.mean(
			np.array([get_embedding(x) for x in nltk.word_tokenize(query.lower())], dtype=float),
			axis=0
    ))

    # Баримтуудын нэр ба ижил төстэй байдлын оноог хадгалах жагсаалт
    rank = []

    # out_dict бүтэц: {баримтын_нэр: баримтын_вектор}
    # Бүх баримтын хувьд асуулга ба тухайн баримтын векторын ижил төстэй байдлыг тооцоолно.
    for k,v in out_dict.items():
			# (баримтын нэр, ижил төстэй байдал)
        	rank.append((k, get_sim(query_words, v)))

    # Ихээс бага руу ижил төсөөт оноог эрэмбэлэх
    rank = sorted(rank, key=lambda t: t[1], reverse=True)
    print('Ranked Documents :')
    return rank

In [ ]:
# Хайлтын асуулга бүхий IR функцийг дуудах
Ranked_documents("cricket")

Ranked Documents :


[('But the man behind the wickets at the other end was watching just as keenly. With an affirmative nod from Dhoni, India captain Rohit Sharma promptly asked for a review. Sure enough, the ball would have clipped the top of middle and leg.',
  [np.float64(0.4495432862154046)]),
 ('He points out that public transport is very good in Mumbai and New Delhi, where there is a good network of suburban and metro rail systems.',
  [np.float64(0.23973446434982593)]),
 ('With the Union cabinet approving the amendments to the Motor Vehicles Act, 2016, those caught for drunken driving will have to have really deep pockets, as the fine payable in court has been enhanced to Rs 10,000 for first-time offenders.',
  [np.float64(0.1832371201201335)]),
 ('Natural language processing (NLP) is an area of computer science and artificial intelligence concerned with the interactions between computers and human (natural) languages, in particular how to program computers to process and analyze large amounts of n

In [ ]:
# Хайлтын асуулга = driving
Ranked_documents("driving")


Ranked Documents :


[('With the Union cabinet approving the amendments to the Motor Vehicles Act, 2016, those caught for drunken driving will have to have really deep pockets, as the fine payable in court has been enhanced to Rs 10,000 for first-time offenders.',
  [np.float64(0.3594728772380066)]),
 ('But the man behind the wickets at the other end was watching just as keenly. With an affirmative nod from Dhoni, India captain Rohit Sharma promptly asked for a review. Sure enough, the ball would have clipped the top of middle and leg.',
  [np.float64(0.19042557572691243)]),
 ('He points out that public transport is very good in Mumbai and New Delhi, where there is a good network of suburban and metro rail systems.',
  [np.float64(0.17066536889810102)]),
 ('Natural language processing (NLP) is an area of computer science and artificial intelligence concerned with the interactions between computers and human (natural) languages, in particular how to program computers to process and analyze large amounts of 

## Гүнзгий сургалтын аргаар текст ангилах

In [ ]:
import pandas as pd
# 'spam.csv' файлыг Pandas ашиглан датафрейм хэлбэрээр унших
file_content = pd.read_csv('spam.csv', encoding = "ISO-8859-1")

# И-мэйл дэх жишээ мэдээллийг шалгах
# Датафреймийн 'v2' баганаас 1-р индексийн мөрийг авах
file_content['v2'][1]


'Ok lar... Joking wif u oni...'

In [ ]:
# NLTK-тэй холбоотой сангууд
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

import re

# Scikit-learn-тэй холбоотой сангууд
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

# График зурах сан
import matplotlib.pyplot as plt

In [ ]:
import nltk
nltk.download('stopwords')
# Зогсоох үгсийг арилгах
stop = stopwords.words('english')

# Датафрейм доторх 'v2' баганын бүх текстээс зогсоох үгсийг арилгах
file_content['v2'] = file_content['v2'].apply(
    lambda x: " ".join(
        x for x in x.split() if x not in stop
    )
)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
# Хэрэгцээтэй баганыг сонгох
Email_Data = file_content[['v1', 'v2']]

# Баганын нэрийг өөрчлөх
Email_Data = Email_Data.rename(columns={"v1": "Target", "v2": "Email"})

# Шинэ датафреймийн эхний хэдэн мөрийг харуулах
Email_Data.head()

,Target,Email
0,ham,"Go jurong point, crazy.. Available bugis n gre..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry 2 wkly comp win FA Cup final tkts 2...
3,ham,U dun say early hor... U c already say...
4,ham,"Nah I think goes usf, lives around though"


In [ ]:
# Текстийг жижиг үсэг рүү хөрвүүлэх
# Цэг, таслал, давхар хоосон зай зэрэг тусгай тэмдэгтүүдийг устгах
Email_Data['Email'] = Email_Data['Email'].apply(
lambda x: re.sub('[!@#$:).;,?&]', '', x.lower())
)

# Давхар хоосон зайг нэг хоосон зайгаар солих
Email_Data['Email'] = Email_Data['Email'].apply(lambda x: re.sub('\s+', ' ', x))

# Эхний 5 мөрийг харуулах
Email_Data['Email'].head(5)

<>:8: SyntaxWarning: invalid escape sequence '\s'
<>:8: SyntaxWarning: invalid escape sequence '\s'
/tmp/ipython-input-1717041694.py:8: SyntaxWarning: invalid escape sequence '\s'
  Email_Data['Email'] = Email_Data['Email'].apply(lambda x: re.sub('\s+', ' ', x))


,Email
0,go jurong point crazy available bugis n great ...
1,ok lar joking wif u oni
2,free entry 2 wkly comp win fa cup final tkts 2...
3,u dun say early hor u c already say
4,nah i think goes usf lives around though


In [ ]:
# Оролтын текст болон зорилтот классуудыг хуваах
list_sentences_rawdata = Email_Data["Email"].fillna("_na_").values

# Зорилтот баганыг тодорхойлох
list_classes = ["Target"]

# ‘Target’ баганыг NumPy массив болгох
target = Email_Data[list_classes].values

# Оролт болон зорилтот классыг нэг датафреймд хадгалах
To_Process=Email_Data[['Email', 'Target']]


In [ ]:
# Өгөгдлийг 80:20 харьцаагаар сургалтын ба туршилтын багцад хуваах
train, test = train_test_split(
			To_Process,
			test_size=0.2	# 20% нь туршилтын өгөгдөл
)

In [ ]:
# Дарааллын урт ба үгийн тоон хязгаар тодорхойлох
# Загварт оруулах өгүүлбэрийн хамгийн их урт (токены тоогоор)
# Хэрэв өгүүлбэр урт бол таслана, богино бол тэгээр нөхнө
MAX_SEQUENCE_LENGTH = 300

# # Өгөгдлийн санд хамгийн их давтагдсан TOP 20,000 үгийг хадгалах
# Үүнээс цааших үгсийг <UNK> (танигдаагүй) гэж тэмдэглэнэ
MAX_NB_WORDS = 20000


In [ ]:
# Текст өгөгдлийг токенжуулж тоон дараалалд хувиргах
from tensorflow.keras.preprocessing.text import Tokenizer

# Tokenizer үүсгэх
tokenizer = Tokenizer(num_words=MAX_NB_WORDS)

# Сургалтын өгөгдөл дээр үгийн сан үүсгэх
tokenizer.fit_on_texts(train.Email)

# Текстийг токен дараалалд хувиргах
train_sequences = tokenizer.texts_to_sequences(train.Email)
test_sequences = tokenizer.texts_to_sequences(test.Email)


In [ ]:
# Үг ба индексийн харьцааны толь үүсгэх
# {үг: индекс} хэлбэрийн dictionary
word_index = tokenizer.word_index

# Корпус дахь нийт өвөрмөц үгийн тоог хэвлэх
print('Өвөрмөц токены тоо: %s' % len(word_index))


Өвөрмөц токены тоо: 8423


In [ ]:
# Текстийн тоон дарааллыг тогтмол урттай болгох (padding)
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Сургалтын өгөгдлийн дарааллыг MAX_SEQUENCE_LENGTH урттай болгоно
train_data = pad_sequences(train_sequences, maxlen=MAX_SEQUENCE_LENGTH)

# Туршилтын өгөгдлийн дарааллыг мөн адил урттай болгоно
test_data = pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH)

In [ ]:
# Сургалтын болон туршилтын өгөгдлийн хэмжээ шалгах
print(train_data.shape)
print(test_data.shape)


(4457, 300)
(1115, 300)


In [ ]:
# Сургалтын болон туршилтын өгөгдлийн шошго (labels) бэлтгэх
train_labels = train['Target']
test_labels = test['Target']


In [ ]:
# Шошгуудыг (labels) тоон утгад хөрвүүлэх
from sklearn.preprocessing import LabelEncoder

# LabelEncoder объект үүсгэх
le = LabelEncoder()

# Сургалтын шошгууд дээр fit хийх
le.fit(train_labels)

# Сургалтын шошгуудыг тоон массив болгон хөрвүүлэх
train_labels = le.transform(train_labels)

# Туршилтын шошгуудыг тоон массив болгон хөрвүүлэх
test_labels = le.transform(test_labels)


In [ ]:
import numpy as np

# LabelEncoder-д тодорхой болсон ангиллуудыг хэвлэх
print("Ангиллууд:", le.classes_)

# Сургалтын өгөгдлийн шошгуудын өвөрмөц утгууд болон давтамжийг шалгах
print(np.unique(train_labels, return_counts=True))

# Туршилтын өгөгдлийн шошгуудын өвөрмөц утгууд болон давтамжийг шалгах
print(np.unique(test_labels, return_counts=True))


Ангиллууд: ['ham' 'spam']
(array([0, 1]), array([3861,  596]))
(array([0, 1]), array([964, 151]))


In [ ]:
# Шошгуудыг one-hot encoding буюу categorical форматад хөрвүүлэх
from tensorflow.keras.utils import to_categorical

# Сургалтын шошгуудыг one-hot форматад хөрвүүлэх
labels_train = to_categorical(np.asarray(train_labels))
# Туршилтын шошгуудыг one-hot форматад хөрвүүлэх
labels_test = to_categorical(np.asarray(test_labels))

# Хэмжээг шалгах
print('Өгөгдлийн тенсор хэлбэр:', train_data.shape)
print('Шошгын тенсор хэлбэр:', labels_train.shape)
print('Шошгын тенсор хэлбэр:', labels_test.shape)


Өгөгдлийн тенсор хэлбэр: (4457, 300)
Шошгын тенсор хэлбэр: (4457, 2)
Шошгын тенсор хэлбэр: (1115, 2)


In [ ]:
# Word embedding-ийн хэмжээ
EMBEDDING_DIM = 100

# Өгүүлбэрийн хамгийн их урт хэвлэх
print(MAX_SEQUENCE_LENGTH)


300


In [ ]:
# Шаардлагатай сангуудыг импортлох
# Ерөнхий Python сангууд
import sys
import os
import re
import csv
import codecs
import numpy as np
import pandas as pd

# TensorFlow Keras-аас текст боловсруулах санг импортлох
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

# TensorFlow Keras давхарга ба моделийн бүрдэл
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import (Dense, Input, LSTM, SimpleRNN,
          Bidirectional, Embedding, Dropout, Activation, Flatten,
			BatchNormalization, Conv1D, MaxPooling1D, GlobalMaxPool1D)

# TensorFlow Keras нэмэлт хэрэгслүүд
from tensorflow.keras import initializers, regularizers, constraints, optimizers, layers


In [ ]:
print('CNN 1D загвар сургах')

model = Sequential()

# Word Embedding давхарга
# MAX_NB_WORDS хүртэлх үгийг EMBEDDING_DIM хэмжээст вектор болгон хувиргах
model.add(Embedding(MAX_NB_WORDS,
                    EMBEDDING_DIM,
                    input_length = MAX_SEQUENCE_LENGTH))

# Завсардалт (Dropout) давхарга: Overfitting-с хамгаалах model.add(Dropout(0.5))

# 1D Convolution давхарга (128 ширхэг шүүлтүүр, kernel size = 5, ReLU идэвхжүүлэлт)
model.add(Conv1D(128, 5, activation = 'relu'))

# MaxPooling давхарга (Feature map-ийг багасгаж, онцлогийг төвлөрүүлэх) model.add(MaxPooling1D(5))

# Завсардалт
model.add(Dropout(0.5))

# Batch Normalization (Давхаргын гаралтыг нормчилж, сургалтын тогтвортой байдлыг нэмэгдүүлэх)
model.add(BatchNormalization())

# Хоёр дахь Conv1D давхарга
model.add(Conv1D(128, 5, activation = 'relu'))
model.add(MaxPooling1D(5))
model.add(Dropout(0.5))
model.add(BatchNormalization())

# Flatten давхарга (Convolution давхаргын гаралтыг 1 хэмжээст вектор болгон хувиргах)
model.add(Flatten())

# Dense давхарга (128 нэгж, ReLU идэвхжүүлэлт)
model.add(Dense(128, activation = 'relu'))

# Output layer (2 ангиллын softmax - магадлалын таамаглал)
model.add(Dense(2, activation = 'softmax'))

# Загварыг компайл хийх
# - Алдааны функц: categorical_crossentropy
# - Оптимизатор: rmsprop
# - Хэмжүүр: accuracy
model.compile(loss = 'categorical_crossentropy',
              optimizer = 'rmsprop',
              metrics = ['acc'])


CNN 1D загвар сургах


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [ ]:
# Загварыг сургалтын өгөгдөл дээр тохируулах
model.fit(
		 # Сургалтын өгөгдөл (токенчлогдсон, padding-тай өгүүлбэрүүд)
    train_data,
    labels_train, 	# Сургалтын шошгууд (one-hot encoding)
    batch_size = 64,    # Нэг удаад боловсруулж буй өгөгдлийн багц хэмжээ
    epochs = 5,         # Эпохийн тоо
   # Validation өгөгдөл: Сургалтын явцад загварын гүйцэтгэлийг үнэлэх, overfitting эсэхийг хянах
    validation_data = (test_data, labels_test)
)


Epoch 1/5
70/70 ━━━━━━━━━━━━━━━━━━━━ 28s 240ms/step - acc: 0.8109 - loss: 0.4888 - val_acc: 0.8646 - val_loss: 0.4684
Epoch 2/5
70/70 ━━━━━━━━━━━━━━━━━━━━ 6s 91ms/step - acc: 0.9736 - loss: 0.0983 - val_acc: 0.8646 - val_loss: 0.6407
Epoch 3/5
70/70 ━━━━━━━━━━━━━━━━━━━━ 7s 98ms/step - acc: 0.9844 - loss: 0.0636 - val_acc: 0.9336 - val_loss: 0.4737
Epoch 4/5
70/70 ━━━━━━━━━━━━━━━━━━━━ 10s 94ms/step - acc: 0.9924 - loss: 0.0313 - val_acc: 0.8807 - val_loss: 0.2668
Epoch 5/5
70/70 ━━━━━━━━━━━━━━━━━━━━ 7s 101ms/step - acc: 0.9960 - loss: 0.0139 - val_acc: 0.9686 - val_loss: 0.3332


In [ ]:
# Туршилтын өгөгдөл дээр таамаглал хийх
predicted = model.predict(test_data)

# Таамагласан үр дүнг хэвлэх
predicted

35/35 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step


array([[0.77999026, 0.2200097 ],
       [0.3169581 , 0.6830419 ],
       [0.3819572 , 0.6180428 ],
       ...,
       [0.70028013, 0.29971993],
       [0.31299052, 0.6870095 ],
       [0.71541935, 0.28458068]], dtype=float32)

In [ ]:
# Загварын үнэлгээ хийхэд шаардлагатай санг импортлох
import sklearn
from sklearn.metrics import precision_recall_fscore_support as score

# precision, recall, fscore, support тооцоолох
precision, recall, fscore, support = score(labels_test, predicted.round())

# Үр дүнг хэвлэх
# Загварын зөв таамагласан эерэг хувь
print('precision: {}'.format(precision))

# Бодит эерэгүүдээс хэдийг нь зөв олсон хувь
print('recall: {}'.format(recall))

# Precision ба Recall-ийн зохицсон дундаж (harmonic mean)
print('fscore: {}'.format(fscore))

# Тухайн ангилалд багтсан жишээнүүдийн тоо
print('support: {}'.format(support))

print("############################")

# Ангиллын тайлан (precision, recall, f1-score, support бүгдийг ангилал тус бүрээр харуулах)
print(sklearn.metrics.classification_report(labels_test, predicted.round()))


precision: [0.96589769 0.99152542]
recall: [0.99896266 0.77483444]
fscore: [0.98215196 0.86988848]
support: [964 151]
############################
              precision    recall  f1-score   support

           0       0.97      1.00      0.98       964
           1       0.99      0.77      0.87       151

   micro avg       0.97      0.97      0.97      1115
   macro avg       0.98      0.89      0.93      1115
weighted avg       0.97      0.97      0.97      1115
 samples avg       0.97      0.97      0.97      1115



In [ ]:
# Шаардлагатай санг импортлох
from tensorflow.keras.layers import SimpleRNN

In [ ]:
# RNN загвар сургах
print('SIMPLERNN загвар сургах')

# Sequential загвар үүсгэх
model = Sequential()

# Embedding давхарга
model.add(Embedding(MAX_NB_WORDS,
           EMBEDDING_DIM,
           input_length = MAX_SEQUENCE_LENGTH
 ))

# SimpleRNN давхарга
model.add(SimpleRNN(2, input_shape = (None, 1)))

# Гаралтын нягт давхарга
model.add(Dense(2, activation = 'softmax'))

# Загварыг компайл хийх
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

# Загварыг сургах
model.fit(train_data, labels_train,
          batch_size=16,
          epochs=5,
          validation_data = (test_data, labels_test))


SIMPLERNN загвар сургах
Epoch 1/5


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


279/279 ━━━━━━━━━━━━━━━━━━━━ 10s 25ms/step - accuracy: 0.8358 - loss: 0.6069 - val_accuracy: 0.9094 - val_loss: 0.4443
Epoch 2/5
279/279 ━━━━━━━━━━━━━━━━━━━━ 6s 21ms/step - accuracy: 0.9504 - loss: 0.3648 - val_accuracy: 0.9175 - val_loss: 0.3545
Epoch 3/5
279/279 ━━━━━━━━━━━━━━━━━━━━ 5s 18ms/step - accuracy: 0.9770 - loss: 0.2240 - val_accuracy: 0.9309 - val_loss: 0.2938
Epoch 4/5
279/279 ━━━━━━━━━━━━━━━━━━━━ 6s 23ms/step - accuracy: 0.9865 - loss: 0.1448 - val_accuracy: 0.9318 - val_loss: 0.2697
Epoch 5/5
279/279 ━━━━━━━━━━━━━━━━━━━━ 5s 18ms/step - accuracy: 0.9938 - loss: 0.0989 - val_accuracy: 0.8897 - val_loss: 0.3713


In [ ]:
# Туршилтын өгөгдөл дээр SimpleRNN загвараар урьдчилан таамаглах
predicted_Srnn = model.predict(test_data)

# Урьдчилан таамагласан үр дүнг харах
predicted_Srnn


35/35 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step


array([[0.99233353, 0.00766654],
       [0.03342834, 0.9665717 ],
       [0.80683285, 0.19316715],
       ...,
       [0.97924703, 0.02075296],
       [0.96737623, 0.03262378],
       [0.9751591 , 0.0248409 ]], dtype=float32)

In [ ]:
# Загварын үнэлгээ хийхэд шаардлагатай санг импортлох
from sklearn.metrics import precision_recall_fscore_support as score

# Туршилтын өгөгдөл дээр таамагласан үр дүнг 4 үзүүлэлтээр гаргах
precision, recall, fscore, support = score(labels_test, predicted_Srnn.round())

# Ангиллын үзүүлэлтүүдийг хэвлэх
print('precision: {}'.format(precision))
print('recall: {}'.format(recall))
print('fscore: {}'.format(fscore))
print('support: {}'.format(support))

print("############################")

# Бүх ангиллын үзүүлэлтийг нэгтгэсэн тайланг хэвлэх
print(sklearn.metrics.classification_report(labels_test, predicted_Srnn.round()))


precision: [0.93756504 0.59090909]
recall: [0.9346473  0.60264901]
fscore: [0.9361039  0.59672131]
support: [964 151]
############################
              precision    recall  f1-score   support

           0       0.94      0.93      0.94       964
           1       0.59      0.60      0.60       151

   micro avg       0.89      0.89      0.89      1115
   macro avg       0.76      0.77      0.77      1115
weighted avg       0.89      0.89      0.89      1115
 samples avg       0.89      0.89      0.89      1115



In [ ]:
from keras.layers import GlobalMaxPooling1D
# Загварыг сургах
print('LSTM загвар сургах')

model = Sequential()

# Үгийн embedding
model.add(Embedding(input_dim=MAX_NB_WORDS, output_dim=EMBEDDING_DIM, input_length=MAX_SEQUENCE_LENGTH))

# LSTM давхарга
model.add(LSTM(units=16, activation='tanh', recurrent_activation='sigmoid', return_sequences=True))
model.add(Dropout(0.2))
model.add(BatchNormalization())

# Global pooling
model.add(GlobalMaxPooling1D())

# Гаралтын давхарга
model.add(Dense(2, activation='softmax'))

# Загварыг компайл хийх
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Сургалт
model.fit(train_data, labels_train,
          batch_size=16,
          epochs=5,
          validation_data=(test_data, labels_test))

LSTM загвар сургах
Epoch 1/5


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


279/279 ━━━━━━━━━━━━━━━━━━━━ 11s 18ms/step - accuracy: 0.8894 - loss: 0.2787 - val_accuracy: 0.8646 - val_loss: 0.2602
Epoch 2/5
279/279 ━━━━━━━━━━━━━━━━━━━━ 4s 14ms/step - accuracy: 0.9943 - loss: 0.0228 - val_accuracy: 0.9758 - val_loss: 0.0928
Epoch 3/5
279/279 ━━━━━━━━━━━━━━━━━━━━ 4s 15ms/step - accuracy: 0.9992 - loss: 0.0050 - val_accuracy: 0.9803 - val_loss: 0.0833
Epoch 4/5
279/279 ━━━━━━━━━━━━━━━━━━━━ 5s 17ms/step - accuracy: 1.0000 - loss: 9.5359e-04 - val_accuracy: 0.9821 - val_loss: 0.0847
Epoch 5/5
279/279 ━━━━━━━━━━━━━━━━━━━━ 4s 14ms/step - accuracy: 1.0000 - loss: 4.6924e-04 - val_accuracy: 0.9830 - val_loss: 0.0776


In [ ]:
# LSTM загвар ашиглан туршилтын өгөгдөл дээр урьдчилан таамаглах
predicted_lstm = model.predict(test_data)
predicted_lstm

35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step


array([[9.9980110e-01, 1.9892272e-04],
       [6.1624398e-04, 9.9938369e-01],
       [5.4146716e-05, 9.9994588e-01],
       ...,
       [9.9946648e-01, 5.3352013e-04],
       [2.8733762e-03, 9.9712664e-01],
       [9.9959463e-01, 4.0539019e-04]], dtype=float32)

In [ ]:
# sklearn сангаас precision, recall, fscore, support тооцох функц импортлох
from sklearn.metrics import precision_recall_fscore_support as score

# LSTM загварын таамаглалын үнэлгээ хийх
precision, recall, fscore, support = score(labels_test, predicted_lstm.round())

# Ангилал тус бүрийн нарийвчлал (precision), сэргээх чадвар (recall),
# F1 оноо (fscore), дэмжлэг (support)-ийг хэвлэх
print('precision: {}'.format(precision))
print('recall: {}'.format(recall))
print('fscore: {}'.format(fscore))
print('support: {}'.format(support))

print("############################")

# Бүх ангиллын дэлгэрэнгүй үнэлгээний тайланг хэвлэх
print(sklearn.metrics.classification_report(labels_test, predicted_lstm.round()))


precision: [0.98067141 1.        ]
recall: [1.         0.87417219]
fscore: [0.9902414  0.93286219]
support: [964 151]
############################
              precision    recall  f1-score   support

           0       0.98      1.00      0.99       964
           1       1.00      0.87      0.93       151

   micro avg       0.98      0.98      0.98      1115
   macro avg       0.99      0.94      0.96      1115
weighted avg       0.98      0.98      0.98      1115
 samples avg       0.98      0.98      0.98      1115



In [ ]:
# LSTM загвар
print('Хоёр чиглэлт LSTM загвар')

model = Sequential()
# Embedding давхарга: үгийг вектор болгон хувиргана
model.add(Embedding(MAX_NB_WORDS,
                    EMBEDDING_DIM,
                    input_length=MAX_SEQUENCE_LENGTH))

# Хоёр чиглэлт LSTM: өгөгдлийг өнгөрсөн ба ирээдүйгээс зэрэг суралцана
model.add(Bidirectional(
    LSTM(16, return_sequences = True, dropout = 0.1, recurrent_dropout = 0.1)
))

# Conv1D: шинж чанарыг илүү нарийвчлан ялгаж авах
model.add(Conv1D(16, kernel_size = 3, padding = "valid", kernel_initializer = "glorot_uniform"))

# GlobalMaxPool1D: онцлог шинжүүдийг нэгтгэж хэмжээ багасгах
model.add(GlobalMaxPool1D())

# Dense + Dropout: илүү өндөр түвшний төлөөлөл суралцуулах
model.add(Dense(50, activation = "relu"))
model.add(Dropout(0.1))

# Гаралтын давхарга: 2 ангилалд softmax функцээр ангилна
model.add(Dense(2,activation = 'softmax'))

# Загварыг компайл хийх
model.compile(loss = 'categorical_crossentropy',
              optimizer = 'adam',
              metrics = ['accuracy'])

# Загварыг сургах
model.fit(train_data, labels_train,
          batch_size = 16,
          epochs = 3,
          validation_data = (test_data, labels_test))


Хоёр чиглэлт LSTM загвар
Epoch 1/3


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


279/279 ━━━━━━━━━━━━━━━━━━━━ 491s 2s/step - accuracy: 0.9106 - loss: 0.2609 - val_accuracy: 0.9830 - val_loss: 0.0743
Epoch 2/3
279/279 ━━━━━━━━━━━━━━━━━━━━ 479s 2s/step - accuracy: 0.9956 - loss: 0.0197 - val_accuracy: 0.9821 - val_loss: 0.0976
Epoch 3/3
279/279 ━━━━━━━━━━━━━━━━━━━━ 476s 2s/step - accuracy: 0.9990 - loss: 0.0026 - val_accuracy: 0.9821 - val_loss: 0.1043


In [ ]:
# Туршилтын өгөгдөл дээр урьдчилан таамаглах
predicted_blstm=model.predict(test_data)
predicted_blstm

35/35 ━━━━━━━━━━━━━━━━━━━━ 12s 332ms/step


array([[9.9999070e-01, 9.3138779e-06],
       [9.2495518e-04, 9.9907506e-01],
       [2.5830375e-06, 9.9999738e-01],
       ...,
       [9.9898845e-01, 1.0115148e-03],
       [2.0056648e-05, 9.9997997e-01],
       [9.9999940e-01, 6.1407559e-07]], dtype=float32)

In [ ]:
# sklearn-ээс ангиллын үнэлгээний функцуудыг импортлох
from sklearn.metrics import precision_recall_fscore_support as score

# Урьдчилан таамагласан болон жинхэнэ шошго ашиглан precision, recall, fscore, support-г тооцох
precision, recall, fscore, support = score(labels_test, predicted_blstm.round())

# Ангилал тус бүрийн precision, recall, fscore, support-г хэвлэх
print('precision: {}'.format(precision))
print('recall: {}'.format(recall))
print('fscore: {}'.format(fscore))
print('support: {}'.format(support))

print("############################")

# Ангиллын бүрэн тайланг хэвлэх
print(sklearn.metrics.classification_report(labels_test, predicted_blstm.round()))


## Дараагийн үг таамаглах загвар

In [1]:
# Pandas санг ашиглан өгөгдлийг унших
import pandas as pd

# 'spam.csv' файлыг уншиж, өгөгдлийг датафреймд хадгалах
file_content = pd.read_csv('spam.csv', encoding = "ISO-8859-1")

# Имэйлийн текст агуулсан багана (v2)-г сонгож, шинэ датафрейм үүсгэх
Email_Data = file_content[[ 'v2']]

# Датафреймийг жагсаалт (list) болгон хувиргах
list_data = Email_Data.values.tolist()

# Жагсаалтыг харуулах
list_data

[['Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...'],
 ['Ok lar... Joking wif u oni...'],
 ["Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's"],
 ['U dun say so early hor... U c already then say...'],
 ["Nah I don't think he goes to usf, he lives around here though"],
 ["FreeMsg Hey there darling it's been 3 week's now and no word back! I'd like some fun you up for it still? Tb ok! XxX std chgs to send, å£1.50 to rcv"],
 ['Even my brother is not like to speak with me. They treat me like aids patent.'],
 ["As per your request 'Melle Melle (Oru Minnaminunginte Nurungu Vettam)' has been set as your callertune for all Callers. Press *9 to copy your friends Callertune"],
 ['WINNER!! As a valued network customer you have been selected to receivea å£900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hour

In [15]:
# Стандарт сангууд
import os        # Файл системтэй ажиллах (фолдер үүсгэх, файлын зам авах гэх мэт)
import sys       # Python орчны мэдээлэл авах, аргументуудтай ажиллах
import time      # Хугацаа хэмжих, хойшлуулах
import re        # Текстийг дүрмийн дагуу боловсруулж хайх
import random    # Санамсаргүй тоо, элементийн сонголт хийх
import collections  # Тогтмол бүтэцтэй өгөгдлийг хянах (Counter, OrderedDict гэх мэт)
import codecs    # Текст файлыг олон кодчлолтой унших, бичих

# Математик ба өгөгдлийн сан
import numpy as np       # Массив, матриц, математик үйлдэл
import pandas as pd      # Датафрейм үүсгэх, боловсруулах
import scipy             # Статистик тооцоо
from scipy import spatial  # Вектор, зай, косинусын төсөө зэрэг spatial функцууд

# Эх хэлний боловсруулалт
from nltk.tokenize import sent_tokenize, word_tokenize  # Текстийг үг ба өгүүлбэрт хуваах
from nltk.tokenize.toktok import ToktokTokenizer      # Илүү нарийвчлалтай токенжуулагч
tokenizer = ToktokTokenizer()

# TensorFlow дээр суурилсан Keras
from tensorflow.keras.models import Sequential  # Загвар үүсгэх
from tensorflow.keras.layers import Dense, Dropout, LSTM  # Төрөл бүрийн давхарга (Dense, Dropout, LSTM)
from tensorflow.keras.callbacks import ModelCheckpoint  # Загварыг хадгалах, сургалтын явцыг хянах

In [16]:
# Жагсаалтыг задлаж бүх элементийг нэг мөр болгох
# from collections import Iterable
from collections.abc import Iterable

def flatten(items):
    """Давхарласан (nested) iterable-үүдийг нэг түвшинд задална"""
    for x in items:
        # Хэрэв x нь iterable (жагсаалт, tuple г.м) ба string эсвэл bytes биш бол
        if isinstance(x, Iterable) and not isinstance(x, (str, bytes)):
            # Рекурсив байдлаар flatten() дуудах
            for sub_x in flatten(x):
                yield sub_x
        else:
            # Хэрэв энгийн элемент байвал шууд буцаана
            yield x

TextData=list(flatten(list_data))

# Жагсаалтын бүх элементийг нийлүүлж нэг мөр болгох
TextData = ''.join(TextData)


In [17]:
# Шаардлагагүй шинэ мөр шилжүүлэх тэмдэгтийг арилгах
TextData = TextData.replace('\n', '')

# Бүх үсгийг жижигрүүлэх
TextData = TextData.lower()

# regex (pattern) тодорхойлох
pattern = r'[^a-zA-z0-9\s]'

# Дээрх үл хүссэн тэмдэгтүүдийг хоосон тэмдэгтээр ('') солих
TextData = re.sub(pattern, '', ''.join(TextData))

In [18]:
# Токен
tokens = tokenizer.tokenize(TextData)

# Токен бүрийн эхлэл, төгсгөлийн хоосон зайг арилгах
tokens = [token.strip() for token in tokens]

In [19]:
# Токен бүрийн давтамжийг тоолох
word_counts = collections.Counter(tokens)

# Нийт хэдэн ялгаатай үг байгааг тоолох
word_c = len(word_counts)
print(word_c)

# Давтамжийн дарааллаар (ихээс багад) ялгаатай үгсийг авах
distinct_words = [x[0] for x in word_counts.most_common()]

# Ялгаатай үгсийг цагаан толгойн дарааллаар эрэмбэлэх
distinct_words_sorted = list(sorted(distinct_words))

13343


In [20]:
# Бүх үгийг индекслэх
word_index = {x: i for i, x in enumerate(distinct_words_sorted)}

# Өгүүлбэрийн уртыг тодорхойлох
sentence_length = 25


In [21]:
# InputData, OutputData жагсаалтыг үүсгэх
# InputData → загварт оруулах өмнөх үгсийн индексүүд
# OutputData → загвараас гарах дараагийн үгийн индекс
InputData = []
OutputData = []

# Цонхны уртыг гулсуулан бүх токен дээр давтах
for i in range(0, word_c - sentence_length, 1):
    # Input буюу загварт оруулах үгийн дараалал (цонх)
    X = tokens[i:i + sentence_length]

    # Output буюу дараагийн үг
    Y = tokens[i + sentence_length]

    # Input-д байгаа үг бүрийг индексээр хувиргаж хадгалах
    InputData.append([word_index[char] for char in X])

    # Output-г индексээр хадгалах
    OutputData.append(word_index[Y])

# Эхний элементийн Input болон Output-ыг шалгах
print (InputData[:1])
print ("\n")
print(OutputData[:1])


[[5086, 12188, 6352, 9095, 3352, 1920, 8506, 5937, 2535, 7886, 5214, 12908, 6541, 4104, 2531, 2997, 11471, 5170, 1595, 12550, 6590, 6316, 12756, 12085, 8495]]


[4292]


In [23]:
import numpy
from tensorflow.keras.utils import to_categorical

# InputData жагсаалтыг 3 хэмжээст массив болгон хувиргах
# X-ийн хэлбэр: [sample тоо, үгийн дарааллын урт, feature = 1]
X = numpy.reshape(InputData, (len(InputData), sentence_length, 1))

# OutputData жагсаалтыг one-hot кодлох
# Y-ийн хэлбэр: [sample тоо, vocabulary_size]
# Тухайн sample-д зөв гарах үгийн байрлал нь 1, бусад нь 0
Y = to_categorical(OutputData)

# Y-г шалгах
Y


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [24]:
# LSTM загвар тодорхойлох
model = Sequential()

# 256 санах ойн нэгж бүхий LSTM давхарга
model.add(LSTM(256, input_shape = (X.shape[1], X.shape[2])))

# Хэт суралцахаас сэргийлэх dropout
model.add(Dropout(0.2))

# Гаралтын давхарга, Y-ийн хэмжээтэй тэнцэх нэгжүүд
model.add(Dense(Y.shape[1], activation='softmax'))

# Загварыг компайл хийх, loss функц нь categorical_crossentropy, оновчлол ADAM
model.compile(loss='categorical_crossentropy', optimizer='adam')

/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [25]:
# Хяналтын цэг тодорхойлох
# file_name_path нь хадгалах файлын нэрийн форматыг зааж өгнө.
# Файлд епох болон loss (алдагдал)-ын утгыг оруулна.
file_name_path = "weights-improvement-{epoch:02d}-{loss:.4f}.keras"

# ModelCheckpoint объектыг үүсгэж, сургалтын явцад хамгийн сайн жинг хадгалах тохиргоо
checkpoint = ModelCheckpoint(file_name_path, monitor='loss', verbose=1, save_best_only=True, mode='min')

# callbacks жагсаалтад checkpoint-ийг нэмэх
callbacks = [checkpoint]

In [26]:
# Загварыг өгөгдөлтэй нийцүүлж сургалт явуулах
# X, Y нь сургалтын оролт ба гаралт өгөгдөл
# callbacks=callbacks нь өмнө тодорхойлсон хяналтын цэгийг сургалтын явцад ашиглах
model.fit(X, Y, epochs=5, batch_size=128, callbacks=callbacks)

Epoch 1/5
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 8.5514
Epoch 1: loss improved from inf to 7.91831, saving model to weights-improvement-01-7.9183.keras
105/105 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - loss: 8.5454
Epoch 2/5
102/105 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 7.0901
Epoch 2: loss improved from 7.91831 to 7.18824, saving model to weights-improvement-02-7.1882.keras
105/105 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - loss: 7.0939
Epoch 3/5
102/105 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 7.0086
Epoch 3: loss improved from 7.18824 to 7.06502, saving model to weights-improvement-03-7.0650.keras
105/105 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 7.0107
Epoch 4/5
102/105 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 6.8668
Epoch 4: loss improved from 7.06502 to 6.92095, saving model to weights-improvement-04-6.9209.keras
105/105 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 6.8688
Epoch 5/5
100/105 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 6.7703
Epoch 5: loss improved from 6.92095 to 6.

In [27]:
# Сүлжээний хамгийн сайн жинг ачаалах
# Өмнө хадгалсан хяналтын цэгийн файлыг зааж өгнө
file_name = "weights-improvement-05-6.8157.keras"

# Жинг ачаалж загварыг сэргээх
model.load_weights(file_name)

# Загварыг дахин компайл хийж, loss болон optimizer-ийг тохируулах
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [28]:
# Санамсаргүй үгийн дараалал сонгох
# numpy.random.randint функц нь 0 ба InputData-ийн урт хооронд санамсаргүй индекс үүсгэнэ
start = numpy.random.randint(0, len(InputData))

# Санамсаргүйгээр сонгогдсон индексээр эх үгийн дарааллыг авах
input_sent = InputData[start]


In [29]:
# Загварын оролт хэлбэрт шилжүүлэх
# input_sent-ийг (1, урт, 1) хэмжээгээр дахин дүрсэлнэ
X = numpy.reshape(input_sent, (1, len(input_sent), 1))

# Загварыг ашиглан дараагийн үгийг таамаглах
predict_word = model.predict(X, verbose = 0)

# Урьдчилан таамагласан үгний хамгийн их магадлалтай индексийг авах
index = numpy.argmax(predict_word)

# Эх үгийн дараалал болон таамагласан дараагийн үгийн индексийг хэвлэх
print(input_sent)
print("\n")
print(index)


[13038, 8573, 10922, 7138, 12085, 10656, 7821, 12085, 3951, 12142, 5750, 10742, 12085, 9470, 549, 7334, 9836, 12085, 630, 7138, 12085, 13096, 10184, 7334, 1221]


13205


In [30]:
# tokens-ийн жагсаалтаас индекс ба үг үүсгэх толь бичиг
word_index_rev = dict((i, c) for i, c in enumerate(tokens))

# Урьдчилан таамагласан үгийн индексийг үг болгон хөрвүүлэх
result = word_index_rev[index]

# Эх үгийн дарааллын бүх индексийг үг болгон хөрвүүлэх
sent_in = [word_index_rev[value] for value in input_sent]

# Эх үгийн дараалал болон таамагласан дараагийн үгийг хэвлэх
print(sent_in)
print("\n")
print(result)

['need', 'jjc', 'from', 'for', 'skyped', 'to', 'later', 'skyped', 'mini', 'text', 'on', 'attached', 'skyped', 'gentle', 'he', 'the', 'wat', 'skyped', 'if', 'for', 'skyped', 'to', 'drivinto', 'the', 'me']


than
